In [1]:
# импортируем библиотеки, классы и функции
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import optuna
from optuna.samplers import RandomSampler

# подавлям вывод сообщений optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# отключаем предупреждения
import warnings
import logging
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', FutureWarning)
logging.getLogger('fbprophet').setLevel(logging.ERROR)

# задаем количество потоков
import os
os.environ['NUMEXPR_MAX_THREADS'] = '8'

In [2]:
# загружаем данные
df = pd.read_csv('Data/online_retail.csv')
df['date'] = pd.to_datetime(df['date'])
df.columns = ['ds', 'y']

In [3]:
# задаем типы гиперпараметров
param_types = {'changepoint_prior_scale': 'float', 
               'seasonality_prior_scale': 'float'}

# задаем границы поиска значений гиперпараметров
bounds = {'changepoint_prior_scale': [0.001, 0.5],
          'seasonality_prior_scale': [0.01, 10]}

In [4]:
# пишем функцию байесовской оптимизации
def objective(trial):
    """
    Обучаем модель Prophet и возвращаем 
    усредненное значение RMSE.
    """
    # отбор параметров
    params = {}
    for param in ['changepoint_prior_scale', 
                  'seasonality_prior_scale']:
        params[param] = trial.suggest_uniform(
            param, 
            bounds[param][0], 
            bounds[param][1])
        
    # обучение
    m = Prophet(yearly_seasonality=4,
                seasonality_mode='additive',
                **params)
    
    m.fit(df)
    
    df_cv = cross_validation(
        m, 
        initial='730 days', 
        period='30 days', 
        horizon='90 days',
        parallel='processes')
    
    df_p = performance_metrics(df_cv, rolling_window=1)
    print(params, f"\nRMSE:{df_p['rmse'].values[0]}\n")
    
    return df_p['rmse'].values[0]

In [5]:
# задаем стартовое значение генератора 
# псевдослучайных чисел
sampler = RandomSampler(seed=10)

# создаем сессию поиска
study = optuna.create_study(sampler=sampler, 
                            direction='minimize')
# выполняем поиск
study.optimize(lambda trial: objective(trial), n_trials=10)
# печатает наилучшую комбинацию гиперпараметров
# и наилучшее значение метрики
print(f"наилучшая комбинация гиперпараметров:\n{study.best_params}")
print(f"наилучшее значение RMSE:\n{study.best_value}")

{'changepoint_prior_scale': 0.38588900099010626, 'seasonality_prior_scale': 0.217311974100421} 
RMSE:473.26328248210035

{'changepoint_prior_scale': 0.3171904692282114, 'seasonality_prior_scale': 7.490550786560732} 
RMSE:469.9151005963841

{'changepoint_prior_scale': 0.24975499913899263, 'seasonality_prior_scale': 2.255718488853168} 
RMSE:466.77364910412217

{'changepoint_prior_scale': 0.09983336951505237, 'seasonality_prior_scale': 7.607701814867598} 
RMSE:456.95489714448837

{'changepoint_prior_scale': 0.08538630744470518, 'seasonality_prior_scale': 0.8925147435983626} 
RMSE:456.2182883391711

{'changepoint_prior_scale': 0.3429945493655308, 'seasonality_prior_scale': 9.534399528487416} 
RMSE:472.29091971697295

{'changepoint_prior_scale': 0.0029701848976293114, 'seasonality_prior_scale': 5.1268007112239085} 
RMSE:452.67067745613974

{'changepoint_prior_scale': 0.4064978598644046, 'seasonality_prior_scale': 6.129135407625587} 
RMSE:476.13056326803917

{'changepoint_prior_scale': 0.361